# SETUP

In [8]:
import dill as pickle
import nltk
nltk.download("wordnet")
from nltk.corpus import wordnet
import gensim
import os
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd
import operator
import re
import gc
from string import punctuation
from nltk.corpus import stopwords
import tensorflow as tf
import tensorflow_text as text
import functools
from collections import defaultdict
from wordcloud import STOPWORDS
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from transformers import BertForSequenceClassification, BertTokenizer
from sklearn.neighbors import KNeighborsClassifier
from transformers import AutoModel, AutoTokenizer
from sklearn.metrics import classification_report 
from torch.nn.functional import pad
from datasets import Dataset
import datasets
from transformers import AutoModelForSequenceClassification
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import EvalPrediction
import transformers
from transformers import TrainingArguments, Trainer

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/rafaellaporto/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# Import Model

In [9]:
labels = ['Ignore', 'alert']
id2label = {0: 'Ignore', 1: 'alert'}
label2id = {'Ignore': 0, 'alert': 1}

In [10]:
# specify the path to the saved model
model_path = 'mymodel'
mymodel = BertForSequenceClassification.from_pretrained(model_path, local_files_only=True, problem_type="multi_label_classification", num_labels=len(labels), id2label=id2label, label2id=label2id)
tokenizer = AutoTokenizer.from_pretrained(model_path, local_files_only=True)

You are using a model of type roberta to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
Some weights of the model checkpoint at mymodel were not used when initializing BertForSequenceClassification: ['roberta.encoder.layer.8.attention.output.LayerNorm.weight', 'roberta.encoder.layer.22.attention.output.dense.weight', 'roberta.encoder.layer.0.attention.self.query.weight', 'roberta.encoder.layer.1.attention.output.LayerNorm.bias', 'roberta.encoder.layer.6.output.dense.weight', 'roberta.encoder.layer.18.output.LayerNorm.weight', 'roberta.encoder.layer.10.output.LayerNorm.weight', 'roberta.encoder.layer.3.attention.self.value.bias', 'roberta.encoder.layer.6.output.LayerNorm.weight', 'roberta.encoder.layer.16.attention.output.dense.bias', 'roberta.encoder.layer.20.attention.output.dense.bias', 'roberta.encoder.layer.19.intermediate.dense.bias', 'roberta.encoder.layer.12.attention.output.LayerNorm.bias', 'roberta.encoder.layer.1

In [23]:
tweet_label = []
tweet_text = []
id2label = {0: 'Ignore', 1: 'alert'}

def predict_tweet(x,mymodel):
    encoding = tokenizer(x, return_tensors="pt")
    encoding = {k: v.to(mymodel.device) for k,v in encoding.items()}
    outputs = mymodel(**encoding)
    logits = outputs.logits
    logits.shape
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(logits.squeeze().cpu())
    return labels[probs.argmax()]

list_tweets = pd.read_csv("sample_tweet_lab0123.csv")
list_tweets = list_tweets["text"].to_list()


In [21]:
list_tweets = pd.read_csv("sample_tweet_lab0123.csv")

In [22]:
list_tweets

,Unnamed: 0,text,target
0,2010,Going to try and sleep now . But live near th...,0.0
1,2687,Assessing damage and checking on friends . Th...,0.0
2,3812,RT @ taeismyybae : I hope everyone is safe ....,0.0
3,6609,DONATE to # Ecuador Red Cross @ CruzRojaEcua...,0.0
4,5828,RT @ SarahKMoser : @ Oxfam is collecting do...,0.0
...,...,...,...
262,159,USGS reports a M1 . 94 # earthquake 5km S...,1.0
263,164,# earthquake ( EMSC ) : MD 2 . 9 ...,1.0
264,170,1 . 94 earthquake occurred 5km S of Volcano ...,1.0
265,171,USGS reports a M1 . 94 # earthquake 5km S...,1.0


In [13]:
import tqdm #progress bar

In [24]:

for x in tqdm.tqdm(list_tweets) :
    predicted_lab = predict_tweet(x, mymodel)
    tweet_label.append(predicted_lab)
    tweet_text.append(x)

100%|██████████| 267/267 [02:29<00:00,  1.79it/s]


In [25]:
df_tweets_labeled = pd.DataFrame({'tweet_text': tweet_text, 'tweet_label':  tweet_label})

In [30]:
df_tweets_labeled

,tweet_text,tweet_label
0,Going to try and sleep now . But live near th...,alert
1,Assessing damage and checking on friends . Th...,alert
2,RT @ taeismyybae : I hope everyone is safe ....,alert
3,DONATE to # Ecuador Red Cross @ CruzRojaEcua...,alert
4,RT @ SarahKMoser : @ Oxfam is collecting do...,alert
...,...,...
262,USGS reports a M1 . 94 # earthquake 5km S...,alert
263,# earthquake ( EMSC ) : MD 2 . 9 ...,alert
264,1 . 94 earthquake occurred 5km S of Volcano ...,alert
265,USGS reports a M1 . 94 # earthquake 5km S...,alert


In [27]:
list_tweets

['Going to try and sleep now .  But live near the water, entirely possible tsunami sirens may go off still Pretty sure itll be ok though .   # eqnz',
 'Assessing damage and checking on friends .  Thoughts with those worse affected .   # eqnz',
 'RT  @ taeismyybae :  I hope everyone is safe .  YA ALLAH MADAD  # earthquake',
 'DONATE to  # Ecuador Red Cross  @ CruzRojaEcuador  @ ammado fundraiser to help earthquake victims by clicking here  Thanks . ',
 'RT  @ SarahKMoser :   @ Oxfam is collecting donations to assist Ecuador after 7 . 8 magnitude earthquake',
 ' @ atizaztariqkhan  @ AlarmaDita  @ farooq_pm ·There is strong appeal from the federal government and the AJK government to provide good facilities to those who were injured in the earthquake .  And Provide money to rebuild those whose homes have been destroyed .   # earthquake',
 'Owner of Chicago  -  Area Gay Bar Admits to Arson Scheme    #  LGBT   |   ',
 ' @ unitlostgaming Sty, please talk with your audience, if somebody can s

In [28]:
df_tweets_labeled["tweet_label"].value_counts()
#previous 
# Ignore           227
# Machine_alert     48
# Human_alert       16
# Name: tweet_label, dtype: int64

alert    267
Name: tweet_label, dtype: int64

In [29]:
list_tweets = pd.read_csv("sample_tweet_lab0123.csv")["target"].value_counts()
list_tweets

0.0    200
1.0     67
Name: target, dtype: int64